This notebook analyzes data from https://github.com/HuthLab/deep-fMRI-dataset. To set up, see instructions in the `deep-fMRI-dataset` folder.

In [1]:
%load_ext autoreload
%autoreload 2
import datasets
import numpy as np
from os.path import join
from encoding.ridge_utils.SemanticModel import SemanticModel
from matplotlib import pyplot as plt
from typing import List
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer
from encoding.feature_spaces import em_data_dir, data_dir, results_dir
from collections import defaultdict
import pandas as pd
import os
from fit_linear_models import get_dsets

### Load some data

In [13]:
dataset = 'trec'
X, y, X_test, y_test = get_dsets(dataset)
print('shapes', len(X), len(X_test))
print('labels', np.unique(y), np.unique(y_test))

Using custom data configuration default
Reusing dataset trec (/home/chansingh/.cache/huggingface/datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)


  0%|          | 0/2 [00:00<?, ?it/s]

Using custom data configuration default
Reusing dataset trec (/home/chansingh/.cache/huggingface/datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)


  0%|          | 0/2 [00:00<?, ?it/s]

shapes 5452 500
labels [0 1 2 3 4 5] [0 1 2 3 4 5]


In [ ]:
pd.DataFrame.from_dict(r).set_index('feats')

,acc,feats_dim
feats,,
eng1000fmri,0.733583,4778


# Load results

In [1]:
save_dir = '/home/chansingh/mntv1/deep-fMRI/results/linear_models'

In [5]:
for fname in os.listdir(save_dir):
    df = pd.read_pickle(join(save_dir, fname))
    print(fname)
    display(df)

rotten_tomatoes.pkl


,acc,feats_dim
feats,,
eng1000fmri,0.740150,1912
eng1000fmri,0.740150,1912
bow,0.751407,16474


emotion.pkl


,acc,feats_dim
feats,,
eng1000fmri,0.628,1912
eng1000fmri,0.628,1912
bow,0.890,15186
